<a href="https://colab.research.google.com/github/Womics/lab_rotation/blob/main/first_workat_labroat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm

# Equivalent of vPow in estimation function.C++
def vPow(base, exp):
    return np.power(base, exp)

# Equivalent of ZeroOne in estimation function.C++
def ZeroOne(x):
    return 1 - x

# Equivalent of ColSums_cpp in C++
def ColSums_cpp(X):
    return np.sum(X, axis=0)

# Equivalent of Pruning_cpp in C++
def Pruning_cpp(West, z):
    n = West.shape[1]
    for j in range(n):
        if z[j] == 1:
            West[:, j][West[:, j] < 0] = 0
        elif z[j] == 0:
            West[:, j][West[:, j] > 0] = 0
    return West

# Equivalent of EstimateConnections_cpp in C++
def EstimateConnections_cpp(Lambda, Eta, nitr):
    n = Lambda.shape[1]
    I = np.eye(n)
    Theta = np.random.rand(n, n)
    np.fill_diagonal(Theta, 0)

    for itr in range(nitr):
      # step1
        West = Lambda @ (I - Theta)
        #step2
        for i in range(n):
            for j in range(n):
                Theta[i, j] = norm.cdf(West[i, j] + Eta[i, j])
        np.fill_diagonal(Theta, 0)
        #step3
        LT = Lambda @ Theta
        Lambda = Lambda - np.diag(np.diag(Lambda)) + np.diag(np.diag(LT))
    return West

# Displaying the defined functions to ensure they are correctly defined
[vPow(np.array([2, 3]), np.array([2, 3])),
 ZeroOne(np.array([0.2, 0.8])),
 ColSums_cpp(np.array([[1, 2], [3, 4]])),
 Pruning_cpp(np.array([[1, -1], [-1, 1]]), np.array([1, 0])),
 EstimateConnections_cpp(np.array([[0.1, 0.3], [0.2, 0.4]]), np.array([[0.05, 0.07], [0.02, 0.06]]), 1)]


[array([ 4, 27]),
 array([0.8, 0.2]),
 array([4, 6]),
 array([[ 1, -1],
        [ 0,  0]]),
 array([[0.08189437, 0.24319524],
        [0.17585916, 0.28639048]])]

In [ ]:
def EstimateConnectionsWithLabel_cpp(Lambda, Eta, z, nitr):
    n = Lambda.shape[1]
    I = np.eye(n)
    Theta = np.random.rand(n, n)
    np.fill_diagonal(Theta, 0)
    West = np.zeros((n, n))

    for _ in range(nitr):
        # Step 1
        West = Lambda @ (I - Theta)

        # Step 2
        West = Pruning_cpp(West, z)

        # Step 3
        for i in range(n):
            for j in range(n):
                Theta[i, j] = norm.cdf(West[i, j] + Eta[i, j])
        np.fill_diagonal(Theta, 0)

        # Step 4
        LT = Lambda @ Theta
        Lambda = Lambda - np.diag(np.diag(Lambda)) + np.diag(np.diag(LT))

    return West

import numpy as np
from scipy.stats import norm

def likelihood_cpp(Lambda, Eta, z, nitr):
    n = Lambda.shape[1]
    I = np.eye(n)
    Theta = np.zeros((n, n))
    West = EstimateConnectionsWithLabel_cpp(Lambda, Eta, z, nitr)

    for i in range(n):
        for j in range(n):
            Theta[i, j] = norm.cdf(West[i, j] + Eta[i, j])

    LT = Lambda @ Theta
    Lambda = Lambda - np.diag(np.diag(Lambda)) + np.diag(np.diag(LT))
    Error = Lambda - West @ np.linalg.inv(I - Theta)

    like = 1
    for i in range(n):
        for j in range(n):
            if Lambda[i, j] < 0:
                continue
            else:
                like *= norm.pdf(Error[i, j], 0, np.sqrt(0.15))

    return like

def gibbs_cpp(Lambda, Eta, alpha, z, ind):
    z1 = z.copy()
    z0 = z.copy()
    z1[ind] = 1
    z0[ind] = 0

    like1 = likelihood_cpp(Lambda, Eta, z1, 10)
    like0 = likelihood_cpp(Lambda, Eta, z0, 10)
    fact1 = np.prod(vPow(alpha, z1) * vPow(1 - alpha, 1 - z1))
    fact0 = np.prod(vPow(alpha, z0) * vPow(1 - alpha, 1 - z0))

    prob = (like1 * fact1) / (like1 * fact1 + like0 * fact0)
    rand = np.random.uniform(0, 1)
    return 1 if rand < prob else 0

def remove_elem(x, idx):
    return np.delete(x, idx)

def sigmoid_cpp(x):
    return 1 / (1 + np.exp(-x))

def sample_z_cpp(n):
    return np.random.choice([0, 1], size=n, p=[0.5, 0.5])

def rbinom_cpp(n, beta):
    return np.random.binomial(1, beta, n)


In [ ]:
import numpy as np

def estimate_excitatory_probabilities_mproj_cpp(Lambda, Eta, States, nitr_em):
    n = Lambda.shape[1]
    nstate = States.shape[0]
    eps = 1e-5

    alpha = np.full(n, 0.5)
    beta = np.zeros(n)
    pre_alpha = alpha.copy()
    p = np.zeros(nstate)
    q = np.zeros(nstate)

    for itr in range(nitr_em):
        for i in range(nstate):
            state = States[i, :]
            like = likelihood_cpp(Lambda, Eta, state, 10)
            fact = np.prod(np.power(alpha, state) * np.power(1 - alpha, 1 - state))
            p[i] = like * fact
        p /= p.sum()

        for j in range(n):
            beta[j] = np.sum(States[:, j] * p)

        for i in range(nstate):
            state = States[i, :]
            q[i] = np.prod(np.power(beta, state) * np.power(1 - beta, 1 - state))

        for j in range(n):
            alpha[j] = np.sum(States[:, j] * q)

        vec_error = alpha - pre_alpha
        error = np.linalg.norm(vec_error)
        if error < eps:
            break
        else:
            pre_alpha = alpha.copy()

    return alpha


In [ ]:
import numpy as np
from scipy.stats import norm

def estimate_excitatory_probabilities_eproj_cpp(Lambda, Eta, States, nitr_em):
    n = Lambda.shape[1]
    nstate = States.shape[0]
    eps = 1e-16

    alpha = np.full(n, 0.5)
    beta = np.full(n, 0.5)
    pre_alpha = alpha.copy()
    C = np.zeros((nstate, n))

    for itr in range(nitr_em):
        for i in range(nstate):
            state = States[i, :]
            for j in range(n):
                estate = np.delete(state, j)
                ebeta = np.delete(beta, j)
                B = np.prod(np.power(ebeta, estate) * np.power(1 - ebeta, 1 - estate))
                z1 = state.copy()
                z0 = state.copy()
                z1[j] = 1
                z0[j] = 0
                like1 = likelihood_cpp(Lambda, Eta, z1, 5)
                like0 = likelihood_cpp(Lambda, Eta, z0, 5)
                fact1 = np.prod(np.power(alpha, z1) * np.power(1 - alpha, 1 - z1))
                fact0 = np.prod(np.power(alpha, z0) * np.power(1 - alpha, 1 - z0))
                A = np.log(like1 + fact1) - np.log(like0 + fact0)
                C[i, j] = B * A
        a = C.sum(axis=0)
        beta = sigmoid(a)

        # Update probabilities q
        q = np.array([np.prod(np.power(beta, States[i, :]) * np.power(1 - beta, 1 - States[i, :])) for i in range(nstate)])

        # Update alpha
        alpha = np.array([np.sum(States[:, j] * q) for j in range(n)])

        # Check convergence
        if np.linalg.norm(alpha - pre_alpha) < 1e-3:
            break
        pre_alpha = alpha.copy()

    return alpha


In [ ]:
def estimate_excitatory_probabilities_mproj_gibbs_cpp(Lambda, Eta, nitr_em, nsamp, excl_ratio):
    n = Lambda.shape[1]
    nexcl = int(round(nsamp * excl_ratio))
    ngibbs = nsamp - nexcl
    eps = 1e-5

    alpha = np.full(n, 0.5)
    pre_alpha = alpha.copy()
    Z1 = np.zeros((nsamp, n))

    for itr in range(nitr_em):
        if itr == 1:
            samp_z = sample_z_cpp(n)
        for i in range(nsamp):
            for j in range(n):
                samp_z[j] = gibbs_cpp(Lambda, Eta, alpha, samp_z, j)
            Z1[i, :] = samp_z.copy()

        subZ1 = Z1[nexcl:, :]
        beta = subZ1.mean(axis=0)

        for i in range(nsamp):
            samp_z = np.random.binomial(1, beta)
            Z2[i, :] = samp_z

        alpha = Z2.mean(axis=0)

        if np.linalg.norm(alpha - pre_alpha) < eps:
            break
        pre_alpha = alpha.copy()

    return alpha


In [ ]:
import numpy as np
from scipy.stats import norm, binom
from numpy.linalg import norm as np_norm

def estimate_excitatory_probabilities_eproj_gibbs_cpp(Lambda, Eta, nitr_em, nsamp, excl_ratio):
    n = Lambda.shape[1]
    nexcl = round(nsamp * excl_ratio)
    ngibbs = nsamp - nexcl
    eps = 1e-16

    Z1 = np.zeros((nsamp, n))
    Z2 = np.zeros((nsamp, n))
    subZ1 = np.zeros((ngibbs, n))
    alpha = np.full(n, 0.5)
    beta = np.full(n, 0.5)
    pre_alpha = alpha.copy()
    C = np.zeros((ngibbs, n))

    for itr in range(nitr_em):
        if itr == 1:
            samp_z = sample_z_cpp(n)
        for i in range(nsamp):
            for j in range(n):
                samp_z[j] = gibbs_cpp(Lambda, Eta, alpha, samp_z, j)
            Z1[i, :] = samp_z.copy()

        subZ1 = Z1[nexcl:, :]

        for i in range(ngibbs):
            state = subZ1[i, :]
            for j in range(n):
                estate = np.delete(state, j)
                ebeta = np.delete(beta, j)
                z1 = state.copy()
                z0 = state.copy()
                z1[j] = 1
                z0[j] = 0
                like1 = likelihood_cpp(Lambda, Eta, z1, 5)
                like0 = likelihood_cpp(Lambda, Eta, z0, 5)
                fact1 = np.prod(np.power(alpha, z1) * np.power(1 - alpha, 1 - z1))
                fact0 = np.prod(np.power(alpha, z0) * np.power(1 - alpha, 1 - z0))
                p1 = like1 * fact1
                p0 = like0 * fact0
                if p1 == 0:
                    p1 = eps
                if p0 == 0:
                    p0 = eps

                C[i, j] = np.log(p1) - np.log(p0)

        a = C.sum(axis=0) / ngibbs
        beta = 1 / (1 + np.exp(-a))

        for i in range(nsamp):
            samp_z = binom.rvs(1, beta)
            Z2[i, :] = samp_z

        alpha = Z2.mean(axis=0)

        vec_error = alpha - pre_alpha
        error = np_norm(vec_error)
        if error < 1e-3:
            break
        pre_alpha = alpha.copy()

    return alpha


In [ ]:
!pip install scipy

# **estimating function.Rの内容をpythonに変換**


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.special import erfinv

def calculate_pre_spike(data, pre_period, post_period):
    L = data.shape[1]  # observed time
    if pre_period == 1:
        pre_spike = data[:, :L - post_period]
    else:
        pre_tmp = data[:, :L - post_period].T
        pre_spike = np.convolve(pre_tmp.flatten(), np.ones(pre_period), mode='valid').reshape(pre_tmp.shape)
        pre_spike[pre_spike > 0] = 1
        pre_spike = pre_spike.T
    return pre_spike

def calculate_post_spike(data, pre_period, post_period):
    L = data.shape[1]  # observed time
    if post_period == 1:
        post_spike = data[:, pre_period:L]
    else:
        post_tmp = data[:, pre_period:L].T
        post_spike = np.convolve(post_tmp.flatten(), np.ones(post_period), mode='valid').reshape(post_tmp.shape)
        post_spike[post_spike > 0] = 1
        post_spike = post_spike.T
    return post_spike

def probit(p):
    return np.sqrt(2) * erfinv(2 * p - 1)

def calculate_spike_prob_matrix(pre_spike, post_spike, nclust):
    L = post_spike.shape[1]
    N = post_spike.shape[0]

    firing_sum = np.sum(pre_spike, axis=1)
    prob_fire = np.zeros((N, N))
    prob_rest = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            Sj = pre_spike[i, :]
            Si = post_spike[j, :]
            S_sum = Si + Sj
            S_dif = Si - Sj
            prob_fire[i, j] = np.sum(S_sum == 2) / firing_sum[i]
            prob_rest[i, j] = np.sum(S_dif == 1) / (L - firing_sum[i])

    prob_fire[prob_fire == 0] = np.finfo(float).eps
    prob_rest[prob_rest == 0] = np.finfo(float).eps

    array_prob = np.zeros((N, N, 2))
    array_prob[:, :, 0] = prob_fire
    array_prob[:, :, 1] = prob_rest

    return array_prob

def estimate_pseudo_connections(prob_fire, prob_rest):
    return np.apply_along_axis(probit, 2, prob_fire) - np.apply_along_axis(probit, 2, prob_rest)

def estimate_connections(Lambda, Eta, z, param):
    N = Lambda.shape[0]
    if z is None:
        West = estimate_connections_cpp(Lambda, Eta, param['number_of_iteration'])
    else:
        West = estimate_connections_with_label_cpp(Lambda, Eta, z, param['number_of_iteration'])
    return West

def estimate_excitatory_label(Lambda, Eta, proj_type, param):
    N = Lambda.shape[1]
    nitr_em = param['number_of_EM_iteration']
    thr = param['excitatory_threshold']

    if proj_type == 'e':
        alpha = estimate_excitatory_probabilities_eproj_cpp(Lambda, Eta, nitr_em)
    elif proj_type == 'm':
        alpha = estimate_excitatory_probabilities_mproj_cpp(Lambda, Eta, nitr_em)

    z = (alpha > thr).astype(int)
    return z

def estimate_excitatory_label_gibbs(Lambda, Eta, proj_type, param):
    N = Lambda.shape[1]
    nalpha = param['number_of_alpha']
    nsamp = param['number_of_Gibbs_sampler']
    excl_ratio = param['gibbs_exclude_ratio']

    Alphas = []
    if proj_type == 'e':
        for i in range(nalpha):
            alpha = estimate_excitatory_probabilities_eproj_gibbs_cpp(Lambda, Eta, param['number_of_EM_iteration'], nsamp, excl_ratio)
            Alphas.append(alpha)
    elif proj_type == 'm':
        for i in range(nalpha):
            alpha = estimate_excitatory_probabilities_mproj_gibbs_cpp(Lambda, Eta, param['number_of_EM_iteration'], nsamp, excl_ratio)
            Alphas.append(alpha)

    z = (np.mean(Alphas, axis=0) != 1).astype(int)
    return z


# **main.Rをpythonで書き直す**

In [ ]:
import pandas as pd
import numpy as np

# Set parameters
pre_period = 10
post_period = 1

params = {
    "excitatory_threshold": 0.5,
    "number_of_iteration": 10,
    "number_of_EM_iteration": 50,
    "number_of_cluster": 10,
    "number_of_alpha": 10,
    "number_of_Gibbs_sampler": 100,
    "gibbs_exclude_ratio": 0.02
}

print("Reading data...")
data = pd.read_pickle(filepath)  # Assuming the data file is a pickle for Python compatibility

print("Preprocessing spikes...")
pre_spike = calculate_pre_spike(data, pre_period, post_period)
post_spike = calculate_post_spike(data, pre_period, post_period)

array_prob = calculate_spike_prob_matrix(pre_spike, post_spike, params['number_of_cluster'])
prob_fire = array_prob[:, :, 0]
prob_rest = array_prob[:, :, 1]
del pre_spike, post_spike  # free memory

Lambda = estimate_pseudo_connections(prob_fire, prob_rest)
Eta = np.apply_along_axis(probit, 2, prob_rest)
del prob_fire, prob_rest  # free memory

print("Estimating excitatory label...")
z = estimate_excitatory_label_gibbs(Lambda, Eta, "e", params)

print("Estimating neural connections...")
West = estimate_connections(Lambda, Eta, z, params)
print("-> done.")


Reading data...


NameError: name 'filepath' is not defined